In [ ]:
# from sklearn.cluster import KMeans 
import pandas as pd
import os
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d.axes3d import Axes3D

import numpy as np
from matplotlib import cm

import csv

import sklearn.cluster
from sklearn.mixture import GaussianMixture
from itertools import combinations
import math


# for interactive plots
%matplotlib qt

# Run Kmeans and get centroids

In [ ]:
#### Names
scenario_names = ['LVM1', 'LVM2', 'LVDAD', 'LVD', 'POV_UnprotectedLeft', 'all']

X1_name = 'alpha_lon_accel_max'
X2_name = 'alpha_lon_brake_max'
X3_name = 'alpha_lon_brake_min'
X4_name = 'response_time'

var_names = [X1_name, X2_name, X3_name, X4_name, 'robustness']

## concat all data

data_concat = pd.DataFrame()
data_all = []

for sce in scenario_names[:-1]:
    scenario = sce
    res_path = "../../results/"+scenario
    data_aux = pd.read_csv(os.path.join(res_path, scenario + '_combined.csv'), index_col=False)
    
    data_all.append(data_aux)
    data_concat = pd.concat([data_concat, data_aux], ignore_index=True)

data_concat.to_csv("../../results/all_combined.csv", index=False)

data_all.append(data_concat)
print(len(data_all))

In [ ]:
fn_centroids_all_scenarios = os.path.join("../../results/", 'global_centroids.csv')

fh_all_centroids = open(fn_centroids_all_scenarios, 'w')
writer_all_centroids = csv.writer(fh_all_centroids)
writer_all_centroids.writerow(var_names)

centroids_all = []

for i, data in enumerate(data_all):
    #### Run Kmeans
    clusters = 3
    model = sklearn.cluster.KMeans(n_clusters = clusters, n_jobs = 4)
    labels = model.fit_predict(data)
    centroids = model.cluster_centers_
    
    scenario = scenario_names[i]
    if scenario == 'all':
        res_path = "../../results/"
    else:
        res_path = "../../results/"+scenario

    print(scenario)
    
    ## Save centorids
    file_name = os.path.join(res_path, scenario + '_centroids.csv')

    print("saving centroids to", file_name)
    
    centroids_aux = centroids.tolist()
    res = [var_names] + centroids_aux
    with open(file_name, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(res)
    
    writer_all_centroids.writerow([scenario])
    writer_all_centroids.writerows(centroids_aux)
    
    ## save data with cluste
    file_name = os.path.join(res_path, scenario + '_param_cluster.csv')
    
    print('Saving labels to', file_name)

    data['cluster_label'] = labels  
    data.to_csv(file_name, index=False)

    #### Run combination of var
    comb_names = combinations([X1_name, X2_name, X3_name, X4_name], 2) 
    names_list = list(comb_names)

    rob = 'robustness'
    R = data[rob]

    for i, var in enumerate(names_list):
        search_name_x = names_list[i][0]
        search_name_y = names_list[i][1]

        X1 = data[search_name_x]
        X2 = data[search_name_y]
        
        ## Plot data
        fig = plt.figure(i)
        ax = plt.axes(projection='3d')    

        ax.scatter3D(X1, X2, R, c=labels, cmap=cm.RdYlGn, alpha=0.8)
        ax.set_xlabel(search_name_x, fontsize=15)
        ax.set_ylabel(search_name_y, fontsize=15)
        ax.set_zlabel('Robustness', fontsize=15)
        plt.tick_params(labelsize=10) 

        figname = os.path.join(res_path, scenario + '_Kmeans_' +  search_name_x + '_vs_' + search_name_y + '.png')

        plt.savefig(figname, dpi=200)

# plt.show()

fh_all_centroids.close()

# Experimental

In [ ]:
scenario_names = ['LVM1', 'LVM2', 'LVDAD', 'LVD', 'POV_UnprotectedLeft']
scenario = scenario_names[1]
# print(scenario)
#
res_path = "../../results/"+scenario

X1_name = 'alpha_lon_accel_max'
X2_name = 'alpha_lon_brake_max'
X3_name = 'alpha_lon_brake_min'
X4_name = 'response_time'

# Read data from a csv to pandas data frame
data = pd.read_csv(os.path.join(res_path, scenario + '_combined.csv'), index_col=False)

In [ ]:
from matplotlib.text import Annotation
from mpl_toolkits.mplot3d.proj3d import proj_transform

class Annotation3D(Annotation):
    def __init__(self, text, xyz, *args, **kwargs):
        super().__init__(text, xy=(0,0), *args, **kwargs)
        self._xyz = xyz

    def draw(self, renderer):
        x2, y2, z2 = proj_transform(*self._xyz, renderer.M)
        self.xy=(x2,y2)
        super().draw(renderer)


def _annotate3D(ax,text, xyz, *args, **kwargs):
    '''Add anotation `text` to an `Axes3d` instance.'''

    annotation= Annotation3D(text, xyz, *args, **kwargs)
    ax.add_artist(annotation)

setattr(Axes3D,'annotate3D',_annotate3D)

In [ ]:
search_name_x = "alpha_lon_brake_min"
search_name_y = 'alpha_lon_brake_max'
rob = 'robustness'
X1 = data[search_name_x]
X2 = data[search_name_y]
R = data[rob]

In [ ]:
## concatenate data from all scenarios
concat_flag = 1

data_all = pd.DataFrame()

data = []
for sce in scenario_names:
    print(sce)
    scenario = sce
    res_path = "../../results/"+scenario
    data_aux = pd.read_csv(os.path.join(res_path, scenario + '_combined.csv'), index_col=False)
    data.append(data_aux)
    
    if concat_flag:
        print("concatenation of all scenarios")
        data_all = pd.concat([data_all, data], ignore_index=True)
        scenario = "all"
        res_path = "../../results/"
        
        data.append(data_all)

# Run Kmeans

In [ ]:

clusters = 3

kmeans = KMeans(n_clusters = clusters, n_jobs=4) 
kmeans.fit(data) 

km_labels = kmeans.labels_
km_centers = kmeans.cluster_centers_

# Create the pandas DataFrame  
km_centers_df = pd.DataFrame(km_centers, columns = [X1_name, X2_name, X3_name, X4_name, 'robustness'])
print(df)
print((km_labels))

data['cluster_label'] = km_labels
data = data.sort_values(by=['cluster_label'])

data.to_csv(os.path.join(res_path, scenario + '_param_cluster.csv'), index=False)

In [ ]:
### Plot kmeans centroids

ax = plt.axes(projection='3d')    
fontsize =15
ax.scatter3D(X1, X2, R, c=R, cmap=cm.RdYlGn, alpha=0.3)
ax.set_xlabel(search_name_x, fontsize=fontsize)
ax.set_ylabel(search_name_y, fontsize=fontsize)
ax.set_zlabel('Robustness', fontsize=fontsize)
plt.tick_params(labelsize=10) 

ax.scatter3D(km_centers_df[search_name_x], km_centers_df[search_name_y], km_centers_df[rob], marker='o', s=1000, c='magenta')

for i, txt in enumerate(km_centers_df[rob]):
    print(txt)
    
#     ax.text(km_centers_df[search_name_x].iloc[i], km_centers_df[search_name_y].iloc[i]
#             , km_centers_df[rob].iloc[i], str(km_centers_df[rob].iloc[i]), color='black')
    
    ax.annotate3D(str(km_centers_df[rob].iloc[i])[:5],
                  (km_centers_df[search_name_x].iloc[i], km_centers_df[search_name_y].iloc[i], km_centers_df[rob].iloc[i]),
                  xytext=(-100,60),
                  textcoords='offset points',
                  bbox=dict(boxstyle="round", fc="lightyellow"),
                  arrowprops = dict(arrowstyle="-|>",ec='black', fc='white', lw=2))
    
    
plt.show()

# Experimental

In [ ]:
from sklearn.decomposition import PCA 

pca = PCA(3) 
model = pca.fit(data) 

pca_data = pd.DataFrame(pca.transform(data)) 

print(model.singular_values_)


from matplotlib import colors as mcolors 
import math 

''' Generating different colors in ascending order of their hsv values '''
colors = list(zip(*sorted((tuple(mcolors.rgb_to_hsv(
    mcolors.to_rgba(color)[:3])), name)
                           for name, color in dict( 
                               mcolors.BASE_COLORS, **mcolors.CSS4_COLORS
                           ).items())))[1] 


# number of steps to taken generate n(clusters) colors 
skips = math.floor(len(colors[5 : -5])/clusters) 
cluster_colors = colors[5 : -5 : skips] 



In [ ]:
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt 
%matplotlib qt

fig = plt.figure() 
ax = fig.add_subplot(111, projection = '3d') 
ax.scatter(pca_data[1], pca_data[2], pca_data[0], 
           c = list(map(lambda label : cluster_colors[label], 
                        kmeans.labels_))) 

str_labels = list(map(lambda label:'% s' % label, kmeans.labels_)) 

# list(map(lambda data1, data2, data3, str_label:
#          ax.text(data1, data2, data3, s = str_label, size = 20,
#                  zorder = 20, color = 'k'), pca_data[0], pca_data[1],
#          pca_data[2], str_labels)) 

plt.show() 
